In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 训练集
app_launch_logs = pd.read_csv('E:/TianChi/CCF用户留存预测挑战赛/datas/app_launch_logs.csv')
user_interaction_data = pd.read_csv('E:/TianChi/CCF用户留存预测挑战赛/datas/user_interaction_data.csv')
user_playback_data = pd.read_csv('E:/TianChi/CCF用户留存预测挑战赛/datas/user_playback_data.csv')
user_portrait_data = pd.read_csv('E:/TianChi/CCF用户留存预测挑战赛/datas/user_portrait_data.csv')
video_related_data = pd.read_csv('E:/TianChi/CCF用户留存预测挑战赛/datas/video_related_data.csv')

# 测试集
test_a = pd.read_csv('E:/TianChi/CCF用户留存预测挑战赛/datas/test_without_label.csv')

<font color=pink>这里主办方给了5张表，主要包括了两个维度的信息，第一个维度是用户和APP交互的信息，第二个维度是用户和Video的交互信息，用户在登录APP的时候，会与若干的Video进行交互，这里是的APP数据和Video的数据可以认为是一个上下级的关系，下面简单的介绍一下主办方给的这5张表。</font>

* 1.User portrait data
    * 这部分数据主要描述了User的个人信息
* 2.App launch logs
    * 这张表是本次比赛的核心表，这里记录的用户和APP交互的历史记录，也是我们后续标签构造的来源
* 3.Video related data
    * 这张表是描述视频相关的信息，这些视频是由用户在登录APP之后观看所得
* 4.User playback data
    * 这张表记录了每个用户观看视频的历史记录，并且记录了每次的观看时间
* 5.User interaction data
    * 这张表记录了每个用户和视频发生交互时的交互方式，和第四张表有一点重复

In [6]:
app_launch_logs

,user_id,launch_type,date
0,10157996,0,129
1,10139583,0,129
2,10277501,0,129
3,10099847,0,129
4,10532773,0,129
...,...,...,...
8493873,10184712,0,145
8493874,10418703,0,145
8493875,10313492,0,145
8493876,10239105,0,145


<font color=gree>本次比赛需要比赛选手预测user在指定时间点后7天有几天登录爱奇艺APP，但是这里主办方并没有给相关标签，这里就需要我们自己通过用户登录APP的日志手动进行构造，构造思路如下：</font>
* 1.从App launch logs中提取出每个用户的登录APP的序列
* 2.我们对所有用户选择一个end_date作为参考点，将end_date后7天的用户等APP的天数作为标签，这样便人为构造了标签，注意这里的end_date的选取有两种情况
    * 第一种情况是用户登录APP的记录过少，如果用户登录APP的最新的天与最早登录APP的天之间少于7天的话，我们认为这种用户的历史记录过少，这里我们对这种用户直接随机选取end_data
    * 第二种情况是用户登录APP的记录较多，如果用户 登录APP的最新的天与最早登录APP的天之间大于7天的话，那么我们认为这类用户的历史行为信息就较为丰富，我们在选取end_data的时候就尽可能在用户的历史信息序列里面进行选取，这样就可以尽可能的利用用户的历史行为信息
* 3.我们在获取的end_date之后便可以统计当前end_date之后7天的用户登录APP的天数，这样便完成的标签的构造